# 使用重复元素的网络（VGG）

AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然AlexNet指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。

本节介绍VGG，它的名字来源于论文作者所在的实验室Visual Geometry Group [1]。VGG提出了可以通过重复使用简单的基础块来构建深度模型的思路。

## VGG块

VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为$3\times 3$的卷积层后接上一个步幅为2、窗口形状为$2\times 2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。我们使用`vgg_block`函数来实现这个基础的VGG块，它可以指定卷积层的数量`num_convs`和输出通道数`num_channels`。

In [ ]:
import tensorflow as tf
print(tf.__version__)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

1. **函数定义**：
   - 函数名为 `vgg_block`
   - 接收两个参数：
     - `num_convs`：卷积层的数量
     - `num_channels`：卷积层的输出通道数

2. **块的结构**：
   - 使用 Sequential 模型创建一个顺序的网络块
   - 包含两种主要层：
     - 多个卷积层
     - 一个最大池化层

3. **卷积层特点**：
   - 使用 3×3 的卷积核
   - 使用 'same' 填充保持输入输出尺寸一致
   - 使用 ReLU 激活函数
   - 重复 num_convs 次

4. **池化层特点**：
   - 使用 2×2 的池化窗口
   - 步长为 2
   - 用于降低特征图的空间维度

5. **实际应用**：
   - 这个块是 VGG 网络的基本构建单元
   - 通过调整 num_convs 和 num_channels 可以构建不同深度和宽度的网络层

这种模块化的设计使得 VGG 网络结构更加清晰和易于理解，同时也便于构建不同版本的 VGG 网络。

In [2]:
def vgg_block(num_convs, num_channels):
    blk = tf.keras.models.Sequential()
    for _ in range(num_convs):
        blk.add(tf.keras.layers.Conv2D(
            num_channels,
            kernel_size=3,
            padding='same',
            activation='relu'
        ))
    blk.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    return blk

## VGG网络

与AlexNet和LeNet一样，VGG网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个`vgg_block`，其超参数由变量`conv_arch`定义。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则跟AlexNet中的一样。

现在我们构造一个VGG网络。它有5个卷积块，前2块使用单卷积层，而后3块使用双卷积层。第一块的输出通道是64，之后每次对输出通道数翻倍，直到变为512。因为这个网络使用了8个卷积层和3个全连接层，所以经常被称为VGG-11。





这段代码实现了VGG网络的完整构建过程，我详细解释如下：

### 1. 网络架构配置
```python
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
```
这个元组定义了VGG网络的卷积层架构：
- 包含5个VGG块
- 每个元组包含两个数值：(卷积层数量, 输出通道数)
- 具体配置：
  - 第一块：1层卷积，64通道
  - 第二块：1层卷积，128通道
  - 第三块：2层卷积，256通道
  - 第四块：2层卷积，512通道
  - 第五块：2层卷积，512通道

### 2. VGG网络构建函数
```python
def vgg(conv_arch)
```

#### 网络构建过程
1. **卷积部分**：
   - 使用Sequential模型作为基础容器
   - 遍历`conv_arch`中的配置
   - 根据配置添加VGG块（使用之前定义的`vgg_block`函数）

2. **全连接部分**：
添加一个包含以下层的Sequential模型：
   - Flatten层：将卷积特征展平
   - 两个4096单元的全连接层：
     - 使用ReLU激活
     - 每个后面跟随0.5概率的Dropout
   - 输出层：
     - 10个单元（对应类别数）
     - 使用sigmoid激活函数

### 技术特点
1. **模块化设计**：
   - 使用`conv_arch`参数化网络结构
   - 便于调整网络深度和宽度

2. **规范化结构**：
   - 遵循VGG的标准设计原则
   - 通道数逐层递增
   - 使用统一的全连接层配置

3. **防过拟合措施**：
   - 在全连接层使用Dropout
   - 使用标准化的激活函数

这个实现展示了VGG网络的典型特征：深度架构、规则的结构设计和有效的正则化策略。

详细计算VGG网络每一层的参数数量：

### 1. 卷积块部分

**第一块 (1层, 64通道)**
- 卷积层: (3×3×1×64) + 64 = 640
- 参数总计: 640

**第二块 (1层, 128通道)**
- 卷积层: (3×3×64×128) + 128 = 73,856
- 参数总计: 73,856

**第三块 (2层, 256通道)**
- 第一层: (3×3×128×256) + 256 = 295,168
- 第二层: (3×3×256×256) + 256 = 590,080
- 参数总计: 885,248

**第四块 (2层, 512通道)**
- 第一层: (3×3×256×512) + 512 = 1,180,160
- 第二层: (3×3×512×512) + 512 = 2,359,808
- 参数总计: 3,539,968

**第五块 (2层, 512通道)**
- 第一层: (3×3×512×512) + 512 = 2,359,808
- 第二层: (3×3×512×512) + 512 = 2,359,808
- 参数总计: 4,719,616

### 2. 全连接部分

**展平层**
- 参数数量: 0

**第一个全连接层**
- 输入维度: 7×7×512 = 25,088
- 参数: (25,088×4096) + 4096 = 102,764,544

**第一个Dropout层**
- 参数数量: 0

**第二个全连接层**
- 参数: (4096×4096) + 4096 = 16,781,312

**第二个Dropout层**
- 参数数量: 0

**输出层**
- 参数: (4096×10) + 10 = 40,970

### 总参数统计
- 卷积层部分: 9,219,328
- 全连接层部分: 119,586,826
- 总参数量: 128,806,154

### 特点分析
1. 参数主要集中在全连接层（约93%）
2. 第一个全连接层占据了最多参数（约80%）
3. 卷积层参数相对较少，但提取了关键特征
4. 这种参数分布也解释了为什么后续的网络架构（如ResNet）倾向于减少全连接层的使用

In [3]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

下面我们实现VGG-11。

In [4]:
def vgg(conv_arch):
    net = tf.keras.models.Sequential()
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    net.add(tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='sigmoid')]))
    return net

net = vgg(conv_arch)

下面构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状。

In [ ]:
X = tf.random.uniform((1,224,224,1))
for blk in net.layers:
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)

可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

## 获取数据和训练模型

In [1]:
# 因为VGG-11计算上比AlexNet更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在Fashion-MNIST数据集上进行训练。
# ratio = 4
# small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
# net = vgg(small_conv_arch)



这是一个用于处理Fashion-MNIST数据集的数据加载器实现，我详细解释其结构和功能：

### 1. 类定义与初始化
**DataLoader类**包含以下主要组件：

#### 初始化函数 (`__init__`)
- 加载Fashion-MNIST数据集
- 数据预处理：
  - 图像归一化到[0,1]区间
  - 添加通道维度
  - 转换数据类型（float32和int32）
- 记录训练集和测试集的样本数量

#### 数据批次获取方法
1. **训练数据获取** (`get_batch_train`)
   - 随机采样指定批次大小的数据
   - 将图像调整为224×224大小（适配VGG网络）
   - 返回处理后的图像和标签

2. **测试数据获取** (`get_batch_test`)
   - 功能类似训练数据获取
   - 用于模型评估阶段

#### 数据可视化方法 (`dataset_explorate`)
- 展示数据集的前10个样本
- 使用matplotlib创建2×5的图像网格
- 显示每个样本的图像和对应类别标签
- 类别包括：T恤、裤子、套衫等10种服装类型

### 2. 可视化配置
- 使用自定义的深度学习样式
- 设置图像大小为10×10
- 去除坐标轴显示
- 使用灰度颜色映射

### 3. 使用示例
- 设置批次大小为128
- 创建DataLoader实例
- 获取一个训练批次
- 打印数据形状
- 显示样本可视化

### 技术特点
1. **数据预处理**：
   - 标准化
   - 维度处理
   - 类型转换

2. **灵活性**：
   - 支持批量数据获取
   - 可调整图像大小
   - 随机采样策略

3. **可视化功能**：
   - 直观展示数据集内容
   - 清晰的标签显示
   - 专业的图像布局

这个实现展示了深度学习中数据处理的标准实践，包括数据预处理、批处理和可视化等关键环节。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
# plt.style.use('seaborn-darkgrid')  # 设置matplotlib样式
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0,axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0,axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]
        
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.train_images[index],224,224,)
        return resized_images.numpy(), self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.test_images[index],224,224,)
        return resized_images.numpy(), self.test_labels[index]

    def dataset_explorate(self):
        """展示数据集前10条数据"""
        class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']  # 标签名称列表
        plt.figure(figsize=(10, 10))
        for i in range(10):
            plt.subplot(2, 5, i + 1)  # 创建2行5列的子图
            plt.imshow(self.train_images[i].squeeze(), cmap='gray')  # 显示图像，去掉通道维度
            plt.title(f'Label: {class_names[self.train_labels[i]]}')  # 显示标签
            plt.axis('off')  # 不显示坐标轴
        plt.show()

batch_size = 128
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)
dataLoader.dataset_explorate()

除了使用了稍大些的学习率，模型训练过程与上一节的AlexNet中的类似。

注：这里省略了训练过程的输出，如果您需要进行训练，请执行train_vgg()函数





这段代码实现了VGG网络的训练配置和训练过程，我详细解释如下：

### 1. 训练函数定义
`train_vgg()` 函数实现了完整的训练流程：
- 设置训练轮数(epoch)为5
- 计算每轮的迭代次数（总样本数/批次大小）
- 使用双层循环进行训练
- 定期保存模型权重

### 2. 模型编译配置
使用 `compile()` 方法设置训练参数：

**优化器配置**：
- 使用SGD（随机梯度下降）优化器
- 学习率：0.05
- 动量：0.0（未使用动量）
- 不使用Nesterov加速

**训练参数**：
- 损失函数：sparse_categorical_crossentropy（适用于整数标签的多分类问题）
- 评估指标：准确率（accuracy）

### 3. 训练过程
1. **批次训练**：
   - 获取一个批次的训练数据
   - 使用`fit()`方法进行训练

2. **权重保存策略**：
   - 每20个批次保存一次权重
   - 在每轮最后一个批次也保存权重
   - 保存格式为h5文件

### 4. 代码特点
1. **模型保护**：
   - 定期保存权重防止训练中断
   - 保留了加载预训练权重的选项（当前被注释）

2. **训练配置**：
   - 使用基础的SGD优化器
   - 采用标准的分类损失函数
   - 包含准确率监控

3. **测试代码**：
   - 包含单批次训练测试
   - 完整训练函数被注释（# train_vgg()）

这个实现展示了深度学习模型训练的标准流程，包括模型配置、训练循环和权重保存等关键环节。

In [ ]:
def train_vgg():
#     net.load_weights("5.7_vgg_weights.h5")
    epoch = 5
    num_iter = dataLoader.num_train//batch_size
    for e in range(epoch):
        for n in range(num_iter):
            x_batch, y_batch = dataLoader.get_batch_train(batch_size)
            net.fit(x_batch, y_batch)
            # 每20个批次保存一次，或者是最后一个批次时保存
            if n % 20 == 0 or n == num_iter - 1:
                net.save_weights("5.7_vgg.weights.h5")
                

net.compile(
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=0.05,
        momentum=0.0,
        nesterov=False
    ),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

x_batch, y_batch = dataLoader.get_batch_train(batch_size)
net.fit(x_batch, y_batch)
# train_vgg()

我们将训练好的参数读入，然后取测试数据计算测试准确率

In [ ]:
net.load_weights("5.7_vgg.weights.h5")

x_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(x_test, y_test, verbose=2)

## 小结

* VGG-11通过5个可以重复使用的卷积块来构造网络。根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型。